In [37]:
import numpy as np

In [38]:
import torch

In [39]:
x = torch.FloatTensor([[1,2,3,4,5,6,7],[3,5,6,7,8,4,5]])
x

tensor([[1., 2., 3., 4., 5., 6., 7.],
        [3., 5., 6., 7., 8., 4., 5.]])

In [40]:
x.unfold(0,2,2)

tensor([[[1., 3.],
         [2., 5.],
         [3., 6.],
         [4., 7.],
         [5., 8.],
         [6., 4.],
         [7., 5.]]])

In [41]:
mock_filter_1 = torch.tensor([[1.0,2,1],[2,4,5],[6,7,8]])
mock_filter_2 = torch.tensor([[1.0,6,7],[1,9,5],[4,7,8]])
mock_filter_3 = torch.tensor([[1.0,3,1],[7,0,5],[9,7,0]])
mock_filter_4 = torch.tensor([[3.0,2,4],[0,4,5],[1,7,0]])

In [42]:
mock_filter_combined = torch.stack([mock_filter_1, mock_filter_2, mock_filter_3, mock_filter_4]) 

In [43]:
mock_filter_combined = torch.stack([mock_filter_combined, mock_filter_combined], dim=1)

In [44]:
mock_filter_combined.shape

torch.Size([4, 2, 3, 3])

In [45]:
print(mock_filter_combined[0])
print(mock_filter_combined[1])
print(mock_filter_combined[2])
print(mock_filter_combined[3])

tensor([[[1., 2., 1.],
         [2., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 1.],
         [2., 4., 5.],
         [6., 7., 8.]]])
tensor([[[1., 6., 7.],
         [1., 9., 5.],
         [4., 7., 8.]],

        [[1., 6., 7.],
         [1., 9., 5.],
         [4., 7., 8.]]])
tensor([[[1., 3., 1.],
         [7., 0., 5.],
         [9., 7., 0.]],

        [[1., 3., 1.],
         [7., 0., 5.],
         [9., 7., 0.]]])
tensor([[[3., 2., 4.],
         [0., 4., 5.],
         [1., 7., 0.]],

        [[3., 2., 4.],
         [0., 4., 5.],
         [1., 7., 0.]]])


In [46]:
filter_groups = mock_filter_combined.unfold(0, 2, 2)

In [47]:
filter_groups.shape

torch.Size([2, 2, 3, 3, 2])

In [48]:
filter_groups[0].permute(3,0,1,2)

tensor([[[[1., 2., 1.],
          [2., 4., 5.],
          [6., 7., 8.]],

         [[1., 2., 1.],
          [2., 4., 5.],
          [6., 7., 8.]]],


        [[[1., 6., 7.],
          [1., 9., 5.],
          [4., 7., 8.]],

         [[1., 6., 7.],
          [1., 9., 5.],
          [4., 7., 8.]]]])

In [49]:
filter_groups[1].permute(3,0,1,2)

tensor([[[[1., 3., 1.],
          [7., 0., 5.],
          [9., 7., 0.]],

         [[1., 3., 1.],
          [7., 0., 5.],
          [9., 7., 0.]]],


        [[[3., 2., 4.],
          [0., 4., 5.],
          [1., 7., 0.]],

         [[3., 2., 4.],
          [0., 4., 5.],
          [1., 7., 0.]]]])

### Testing the activations

In [50]:
import torch.nn.functional as F

In [51]:
filters = mock_filter_combined
filters.shape

torch.Size([4, 2, 3, 3])

In [52]:
inputs = torch.randn(2,2,4,4)

In [53]:
activations = F.conv2d(inputs,filters)

In [54]:
activations.shape

torch.Size([2, 4, 2, 2])

In [55]:
inputs

tensor([[[[ 0.6270, -0.7832, -1.2495, -0.7499],
          [-0.8693,  2.7487, -2.8387,  1.1427],
          [ 0.1583, -0.2746, -0.1172,  2.5650],
          [-0.5957, -0.8339, -2.0637, -1.3005]],

         [[-0.7539,  0.1693, -0.3319, -0.5541],
          [ 0.7425, -0.9582, -0.2327, -0.2746],
          [-0.7196,  1.2894, -0.0181, -1.2182],
          [-2.1625,  0.1844,  0.2063,  1.1829]]],


        [[[-0.5855,  0.1040,  0.8715,  0.5336],
          [ 0.0421,  2.0369, -0.0984,  0.1560],
          [-1.5450, -0.9842, -2.0298, -0.3024],
          [-0.8636,  2.4248, -0.7044, -2.2384]],

         [[-1.0185, -0.1035, -0.5811, -0.1009],
          [-2.0223, -0.1259, -2.1468, -0.3971],
          [ 0.4881,  3.4474, -0.6909,  0.6374],
          [ 0.3464,  1.3410,  0.2889,  2.8074]]]])

In [56]:
activations

tensor([[[[ -8.7306,   6.4719],
          [-33.3123, -13.1013]],

         [[-10.4730, -26.8553],
          [-33.4269, -20.5703]],

         [[-17.7423,  18.3985],
          [-31.8027, -11.5654]],

         [[ -9.5868, -18.0975],
          [-13.0067,  -4.7575]]],


        [[[-19.7205,  -6.9364],
          [ 13.6655,  17.1370]],

         [[ -4.3275, -21.2414],
          [ 22.2456, -16.8888]],

         [[-18.6693,  16.5984],
          [  2.2120,  44.8360]],

         [[  8.9538, -24.4551],
          [ 10.9935,  -8.0725]]]])

In [57]:
groups = torch.split(activations, 2, dim=1)

In [58]:
groups

(tensor([[[[ -8.7306,   6.4719],
           [-33.3123, -13.1013]],
 
          [[-10.4730, -26.8553],
           [-33.4269, -20.5703]]],
 
 
         [[[-19.7205,  -6.9364],
           [ 13.6655,  17.1370]],
 
          [[ -4.3275, -21.2414],
           [ 22.2456, -16.8888]]]]), tensor([[[[-17.7423,  18.3985],
           [-31.8027, -11.5654]],
 
          [[ -9.5868, -18.0975],
           [-13.0067,  -4.7575]]],
 
 
         [[[-18.6693,  16.5984],
           [  2.2120,  44.8360]],
 
          [[  8.9538, -24.4551],
           [ 10.9935,  -8.0725]]]]))

In [59]:
groups[0]

tensor([[[[ -8.7306,   6.4719],
          [-33.3123, -13.1013]],

         [[-10.4730, -26.8553],
          [-33.4269, -20.5703]]],


        [[[-19.7205,  -6.9364],
          [ 13.6655,  17.1370]],

         [[ -4.3275, -21.2414],
          [ 22.2456, -16.8888]]]])

### Another test case where filtere are not divisible by the number of groups

In [60]:
mock_filter_1 = torch.tensor([[1.0,2,1],[2,4,5],[6,7,8]])
mock_filter_2 = torch.tensor([[1.0,6,7],[1,9,5],[4,7,8]])
mock_filter_3 = torch.tensor([[1.0,3,1],[7,0,5],[9,7,0]])
mock_filter_4 = torch.tensor([[3.0,2,4],[0,4,5],[1,7,0]])
mock_filter_5 = torch.tensor([[3.0,2,4],[0,4,5],[1,7,0]])

In [61]:
mock_filter_combined = torch.stack([mock_filter_1, mock_filter_2, mock_filter_3, mock_filter_4,mock_filter_5]) 

In [62]:
mock_filter_combined = torch.stack([mock_filter_combined, mock_filter_combined], dim=1)
mock_filter_combined.shape

torch.Size([5, 2, 3, 3])

In [63]:
inputs = torch.randn(2,2,4,4)

In [64]:
activations = F.conv2d(inputs,mock_filter_combined)
activations.shape

torch.Size([2, 5, 2, 2])

In [65]:
inputs

tensor([[[[ 1.1008,  1.2091,  0.2950,  0.6933],
          [-0.8948,  0.0367, -0.3541,  0.8272],
          [ 0.5888, -0.7425, -0.2324,  1.5268],
          [ 0.4478,  0.1243, -0.9328, -0.3839]],

         [[-0.9310, -0.8180,  0.9963,  0.2552],
          [ 1.0000, -1.7174,  1.6753, -1.8214],
          [ 0.2643, -0.8004, -0.1208, -0.3603],
          [ 1.3717, -1.4047,  0.4783,  0.8819]]],


        [[[ 0.2276, -1.4880,  0.6828,  2.2028],
          [-0.6867,  0.7186, -0.3461,  1.6830],
          [ 1.0394, -0.1433, -0.1801,  1.4743],
          [ 0.6210,  0.4716,  0.5496,  0.8919]],

         [[-1.0351, -0.2198,  1.0137, -1.1162],
          [ 1.6118, -0.8813,  1.3146, -1.4627],
          [-1.2326,  1.6830,  1.3244,  0.4763],
          [ 1.8395, -0.4137,  0.6771,  1.3594]]]])

In [66]:
activations

tensor([[[[ -6.1708,  -1.5236],
          [ -9.8483,  -5.5783]],

         [[ -7.0740,  20.7048],
          [-20.8505,  -3.9211]],

         [[  6.8540, -27.8816],
          [  8.0026, -18.3840]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 22.2881,  40.2754],
          [ 38.0442,  46.3490]],

         [[ 24.2835,  55.5061],
          [ 46.1751,  55.6113]],

         [[ 16.1214,  26.2990],
          [ 28.3241,  32.6018]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]])

In [67]:
groups = torch.split(activations, 2, dim=1)
list(groups)

[tensor([[[[ -6.1708,  -1.5236],
           [ -9.8483,  -5.5783]],
 
          [[ -7.0740,  20.7048],
           [-20.8505,  -3.9211]]],
 
 
         [[[ 22.2881,  40.2754],
           [ 38.0442,  46.3490]],
 
          [[ 24.2835,  55.5061],
           [ 46.1751,  55.6113]]]]), tensor([[[[  6.8540, -27.8816],
           [  8.0026, -18.3840]],
 
          [[ -3.6077,   3.8480],
           [-12.8417,  -6.4187]]],
 
 
         [[[ 16.1214,  26.2990],
           [ 28.3241,  32.6018]],
 
          [[ 15.7239,  17.1413],
           [ 21.0698,  25.3058]]]]), tensor([[[[ -3.6077,   3.8480],
           [-12.8417,  -6.4187]]],
 
 
         [[[ 15.7239,  17.1413],
           [ 21.0698,  25.3058]]]])]

In [68]:
groupwise_mean = [torch.mean(groups[i]) for i in range(0,2)]

In [69]:
groupwise_mean

[tensor(18.3919), tensor(8.2599)]

In [70]:
groupwise_std = [torch.std(groups[i]) for i in range(0,2)]
groupwise_std

[tensor(26.1979), tensor(17.9086)]

### Test channelwise receptive field calcualtions

In [71]:
def calculate_receptive_field_layer_no_batch_norm(feature_maps, linear_map_param_1, 
                                                  linear_map_param_2):
    number_of_maps = feature_maps[1]
    std_map_wise = feature_maps.transpose(0,1).contiguous().view(number_of_maps, -1).std(1)
    receptive_fields = feature_maps.transpose(1,3).div(std_map_wise).transpose(1,3)
    receptive_fields = torch.sigmoid(linear_map_param_1 * receptive_fields + linear_map_param_2)
    return receptive_fields

In [72]:
calculate_receptive_field_layer_no_batch_norm(activations)
print(activations.shape)

TypeError: calculate_receptive_field_layer_no_batch_norm() missing 2 required positional arguments: 'linear_map_param_1' and 'linear_map_param_2'

In [74]:
kk=activations.transpose(0,1)
print(kk.shape)
c =kk.contiguous().view(5, -1)

torch.Size([5, 2, 2, 2])


In [75]:
c.shape

torch.Size([5, 8])

In [76]:
B = torch.mean(c,1)
c.std(1)

tensor([23.8029, 29.7446, 22.0265, 14.1724, 14.1724])

In [77]:
print(activations)
print(activations.shape)

tensor([[[[ -6.1708,  -1.5236],
          [ -9.8483,  -5.5783]],

         [[ -7.0740,  20.7048],
          [-20.8505,  -3.9211]],

         [[  6.8540, -27.8816],
          [  8.0026, -18.3840]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 22.2881,  40.2754],
          [ 38.0442,  46.3490]],

         [[ 24.2835,  55.5061],
          [ 46.1751,  55.6113]],

         [[ 16.1214,  26.2990],
          [ 28.3241,  32.6018]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]])
torch.Size([2, 5, 2, 2])


In [78]:
kk=activations.transpose(1,3)
print(kk)
print(kk.shape)
activations.transpose(1,3).div(torch.Tensor([0.1,1,1,1,1])).transpose(1,3)

tensor([[[[ -6.1708,  -7.0740,   6.8540,  -3.6077,  -3.6077],
          [ -9.8483, -20.8505,   8.0026, -12.8417, -12.8417]],

         [[ -1.5236,  20.7048, -27.8816,   3.8480,   3.8480],
          [ -5.5783,  -3.9211, -18.3840,  -6.4187,  -6.4187]]],


        [[[ 22.2881,  24.2835,  16.1214,  15.7239,  15.7239],
          [ 38.0442,  46.1751,  28.3241,  21.0698,  21.0698]],

         [[ 40.2754,  55.5061,  26.2990,  17.1413,  17.1413],
          [ 46.3490,  55.6113,  32.6018,  25.3058,  25.3058]]]])
torch.Size([2, 2, 2, 5])


tensor([[[[-61.7085, -15.2364],
          [-98.4825, -55.7826]],

         [[ -7.0740,  20.7048],
          [-20.8505,  -3.9211]],

         [[  6.8540, -27.8816],
          [  8.0026, -18.3840]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[222.8805, 402.7539],
          [380.4422, 463.4900]],

         [[ 24.2835,  55.5061],
          [ 46.1751,  55.6113]],

         [[ 16.1214,  26.2990],
          [ 28.3241,  32.6018]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]])

In [79]:
receptive_fields = 2.0 * activations + 1.0
print(receptive_fields)

tensor([[[[-11.3417,  -2.0473],
          [-18.6965, -10.1565]],

         [[-13.1479,  42.4096],
          [-40.7011,  -6.8422]],

         [[ 14.7080, -54.7632],
          [ 17.0052, -35.7680]],

         [[ -6.2155,   8.6960],
          [-24.6834, -11.8375]],

         [[ -6.2155,   8.6960],
          [-24.6834, -11.8375]]],


        [[[ 45.5761,  81.5508],
          [ 77.0884,  93.6980]],

         [[ 49.5669, 112.0123],
          [ 93.3503, 112.2226]],

         [[ 33.2427,  53.5979],
          [ 57.6482,  66.2036]],

         [[ 32.4477,  35.2825],
          [ 43.1396,  51.6116]],

         [[ 32.4477,  35.2825],
          [ 43.1396,  51.6116]]]])


In [80]:
receptive_fields = torch.sigmoid(receptive_fields)
print(receptive_fields)

tensor([[[[1.1868e-05, 1.1433e-01],
          [7.5895e-09, 3.8820e-05]],

         [[1.9495e-06, 1.0000e+00],
          [2.1074e-18, 1.0666e-03]],

         [[1.0000e+00, 1.6468e-24],
          [1.0000e+00, 2.9253e-16]],

         [[1.9942e-03, 9.9983e-01],
          [1.9060e-11, 7.2283e-06]],

         [[1.9942e-03, 9.9983e-01],
          [1.9060e-11, 7.2283e-06]]],


        [[[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]],

         [[1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00]]]])


In [81]:
receptive_fields.shape
receptive_fields= receptive_fields.transpose(1, 3).add(torch.Tensor([0.1,1,1,1,2])).transpose(1,3)

In [82]:
print(receptive_fields)

tensor([[[[0.1000, 0.2143],
          [0.1000, 0.1000]],

         [[1.0000, 2.0000],
          [1.0000, 1.0011]],

         [[2.0000, 1.0000],
          [2.0000, 1.0000]],

         [[1.0020, 1.9998],
          [1.0000, 1.0000]],

         [[2.0020, 2.9998],
          [2.0000, 2.0000]]],


        [[[1.1000, 1.1000],
          [1.1000, 1.1000]],

         [[2.0000, 2.0000],
          [2.0000, 2.0000]],

         [[2.0000, 2.0000],
          [2.0000, 2.0000]],

         [[2.0000, 2.0000],
          [2.0000, 2.0000]],

         [[3.0000, 3.0000],
          [3.0000, 3.0000]]]])


In [83]:
print(receptive_fields.shape)

torch.Size([2, 5, 2, 2])


In [92]:
groups = torch.split(activations, 2, dim=1)
groups = list(groups)
print(groups)

[tensor([[[[ -6.1708,  -1.5236],
          [ -9.8483,  -5.5783]],

         [[ -7.0740,  20.7048],
          [-20.8505,  -3.9211]]],


        [[[ 22.2881,  40.2754],
          [ 38.0442,  46.3490]],

         [[ 24.2835,  55.5061],
          [ 46.1751,  55.6113]]]]), tensor([[[[  6.8540, -27.8816],
          [  8.0026, -18.3840]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 16.1214,  26.2990],
          [ 28.3241,  32.6018]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]]), tensor([[[[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]])]


### Test regularizer on activations in layer

#### calculate the numerator

In [264]:
print(groups[0].shape)
print(groups[0])

torch.Size([2, 2, 2, 2])
tensor([[[[ -6.1708,  -1.5236],
          [ -9.8483,  -5.5783]],

         [[ -7.0740,  20.7048],
          [-20.8505,  -3.9211]]],


        [[[ 22.2881,  40.2754],
          [ 38.0442,  46.3490]],

         [[ 24.2835,  55.5061],
          [ 46.1751,  55.6113]]]])


In [265]:
difference_tensor = torch.sub(groups[0], groups[0][0][0])
difference_tensor_2 = torch.sub(groups[1], groups[1][0][0])
print(difference_tensor.shape)

torch.Size([2, 2, 2, 2])


In [266]:
print(difference_tensor.norm(1))
print(difference_tensor_2.norm(1))


tensor(410.5658)
tensor(320.4058)


In [267]:
difference_tensor

tensor([[[[  0.0000,   0.0000],
          [  0.0000,   0.0000]],

         [[ -0.9031,  22.2285],
          [-11.0023,   1.6572]]],


        [[[ 28.4589,  41.7990],
          [ 47.8925,  51.9273]],

         [[ 30.4543,  57.0298],
          [ 56.0234,  61.1896]]]])

In [306]:
input = torch.empty(2)
l1_norms = torch.zeros_like(input)
batch_size = 2
for i in range(0, 2):
    current_group = groups[i]
    norm = torch.zeros(1)
    
    for j in range(0, 2):
        difference_tensor = torch.sub(current_group[j], current_group[j][0])
        print(difference_tensor)
        norm = norm  + difference_tensor.norm(1)
    
    #difference_tensor = torch.sub(current_group, current_group[0][0])
    l1_norms[i] = norm

tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000]],

        [[ -0.9031,  22.2285],
         [-11.0023,   1.6572]]])
tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 1.9954, 15.2308],
         [ 8.1309,  9.2623]]])
tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000]],

        [[-10.4618,  31.7296],
         [-20.8443,  11.9652]]])
tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.3975, -9.1577],
         [-7.2543, -7.2960]]])


In [269]:
l1_norms

tensor([70.4105, 99.1064])

In [270]:
current_group

tensor([[[[  6.8540, -27.8816],
          [  8.0026, -18.3840]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 16.1214,  26.2990],
          [ 28.3241,  32.6018]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]])

In [271]:
current_group[0][0].shape

torch.Size([2, 2])

In [272]:
difference_tensor

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.3975, -9.1577],
         [-7.2543, -7.2960]]])

### calculate with denominator

In [273]:
print(groups[0].shape)
print(groups)

torch.Size([2, 2, 2, 2])
[tensor([[[[ -6.1708,  -1.5236],
          [ -9.8483,  -5.5783]],

         [[ -7.0740,  20.7048],
          [-20.8505,  -3.9211]]],


        [[[ 22.2881,  40.2754],
          [ 38.0442,  46.3490]],

         [[ 24.2835,  55.5061],
          [ 46.1751,  55.6113]]]]), tensor([[[[  6.8540, -27.8816],
          [  8.0026, -18.3840]],

         [[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 16.1214,  26.2990],
          [ 28.3241,  32.6018]],

         [[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]]), tensor([[[[ -3.6077,   3.8480],
          [-12.8417,  -6.4187]]],


        [[[ 15.7239,  17.1413],
          [ 21.0698,  25.3058]]]])]


In [308]:
input = torch.empty(2)
l1_norms = torch.zeros_like(input)
for i in range(0, 2):
    current_group = groups[i]
    # print(current_group.size())
    current_norm = torch.zeros(1)
    for j in range(0, 2):
        current_map_set = current_group[j]  
        print(current_map_set.shape)
        num = current_group.size(0)
        # print(num)
        sub_tensor = current_map_set.contiguous().view(num, -1)
        # print(sub_tensor.shape)
        sub_tensor_2 = current_map_set[0].view(-1,4)
        # print(sub_tensor_2.shape)
        difference_tensor = torch.sub(sub_tensor, sub_tensor_2)
        print(difference_tensor)
        num_tensor = difference_tensor.norm(1, dim=1)
        denom_tensor = torch.add(sub_tensor.norm(1, dim=1), sub_tensor_2.norm(1))
        value_tensor = torch.div(num_tensor, denom_tensor).norm(1)
        current_norm = current_norm + value_tensor
    l1_norms[i] = current_norm

torch.Size([2, 2, 2])
tensor([[  0.0000,   0.0000,   0.0000,   0.0000],
        [ -0.9031,  22.2285, -11.0023,   1.6572]])
torch.Size([2, 2, 2])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.9954, 15.2308,  8.1309,  9.2623]])
torch.Size([2, 2, 2])
tensor([[  0.0000,   0.0000,   0.0000,   0.0000],
        [-10.4618,  31.7296, -20.8443,  11.9652]])
torch.Size([2, 2, 2])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.3975, -9.1577, -7.2543, -7.2960]])


In [309]:
print(sub_tensor)

tensor([[16.1214, 26.2990, 28.3241, 32.6018],
        [15.7239, 17.1413, 21.0698, 25.3058]])


In [310]:
print(denom_tensor)

tensor([206.6925, 182.5870])


In [311]:
print(num_tensor)

tensor([ 0.0000, 24.1055])


In [312]:
l1_norms

tensor([0.5784, 0.9859])

In [313]:
l1_norms

tensor([0.5784, 0.9859])

In [314]:
difference_tensor

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.3975, -9.1577, -7.2543, -7.2960]])

In [315]:
difference_tensor.norm(1, dim=1)

tensor([ 0.0000, 24.1055])

### Activation regularizer across a pair of layer

In [177]:
# our first set of activations are : <activations>
#create second set of activations
inputs = torch.randn(2,2,5,5)
activations_set2 = F.conv2d(inputs,mock_filter_combined)
activations_set2.shape

torch.Size([2, 5, 3, 3])

In [178]:
print(activations.shape)

torch.Size([2, 5, 2, 2])


In [193]:
maps_per_group = int(len(activations_set2[1]) / 2)
#print(maps_per_group)
activation_groups = torch.split(activations_set2, 2, dim=1)
activation_groups = list(activation_groups)

In [194]:
activation_groups

[tensor([[[[ -1.3348,  -8.6145,   8.6003],
           [-30.6904, -23.3993, -16.8704],
           [  6.9054, -13.7017, -10.1715]],
 
          [[ 13.9408, -12.4795,  15.8232],
           [-30.1931, -26.1316,  -1.5836],
           [ -4.8341, -30.3453,  -7.3337]]],
 
 
         [[[-20.0745, -36.9113, -22.3157],
           [  1.5723,   6.4226,  26.4029],
           [-20.2081, -10.8876,  10.6963]],
 
          [[-22.8600, -41.2298, -18.6144],
           [  0.4209,  -7.9651,  15.0301],
           [-29.0653, -12.9288,   8.3575]]]]),
 tensor([[[[ -3.2511,  -0.3013,  -0.8952],
           [ -4.3069, -25.3189, -23.0628],
           [  1.2393,  11.0749, -25.8167]],
 
          [[  1.2795,  -7.4962,  14.2076],
           [-10.3752, -12.2399,   3.0166],
           [  0.0946, -12.5814, -14.0192]]],
 
 
         [[[-32.1756, -12.2432, -16.3602],
           [-19.4285, -12.7395,  22.1246],
           [-10.2177, -30.6002,   7.4415]],
 
          [[ -3.7118, -18.4657, -11.5817],
           [-18.2931, -12.

In [210]:
maps_per_group = int(len(activations[1]) / 2)
#print(maps_per_group)
activation_groups_layer_1 = torch.split(activations, 2, dim=1)
activation_groups_layer_1 = list(activation_groups_layer_1)

In [211]:
activation_groups_layer_1

[tensor([[[[ -6.1708,  -1.5236],
           [ -9.8483,  -5.5783]],
 
          [[ -7.0740,  20.7048],
           [-20.8505,  -3.9211]]],
 
 
         [[[ 22.2881,  40.2754],
           [ 38.0442,  46.3490]],
 
          [[ 24.2835,  55.5061],
           [ 46.1751,  55.6113]]]]), tensor([[[[  6.8540, -27.8816],
           [  8.0026, -18.3840]],
 
          [[ -3.6077,   3.8480],
           [-12.8417,  -6.4187]]],
 
 
         [[[ 16.1214,  26.2990],
           [ 28.3241,  32.6018]],
 
          [[ 15.7239,  17.1413],
           [ 21.0698,  25.3058]]]]), tensor([[[[ -3.6077,   3.8480],
           [-12.8417,  -6.4187]]],
 
 
         [[[ 15.7239,  17.1413],
           [ 21.0698,  25.3058]]]])]

In [205]:
input = torch.empty(2)
groupwise_activation_norms = torch.zeros_like(input)

In [316]:
for i in range(0, 2):
    current_group = activation_groups[i]
    current_group_layer_1 = activation_groups_layer_1[i]
    
    num = current_group.size(0) * current_group.size(1)
    size_of_map = current_group.size(2) * current_group.size(3)
    size_of_map_layer_1 = current_group_layer_1.size(2) * current_group_layer_1.size(3)
    
    map_per_row_view = current_group.contiguous().view(num, -1)       # shape: (Batches*Channels) X (Height * Width)
    first_map_row_view =  current_group_layer_1[0][0].view(-1, size_of_map_layer_1)  # shape: 1 X (Height * Width)
    
    difference_tensor = torch.sub(map_per_row_view, first_map_row_view)
    numerator_tensor = difference_tensor.norm(1, dim=1)
    denominator_tensor = torch.add(map_per_row_view.norm(1, dim=1), first_map_row_view.norm(1))
    groupwise_activation_norms[i] = torch.div(numerator_tensor, denominator_tensor).sum()

RuntimeError: The size of tensor a (9) must match the size of tensor b (4) at non-singleton dimension 1

In [215]:
activation_groups[i].shape

torch.Size([2, 2, 3, 3])

In [216]:
activation_groups_layer_1[i].shape

torch.Size([2, 2, 2, 2])

In [217]:
activation_groups_layer_1[i][0][0].unsqueeze(0)

torch.Size([2, 2])